<a href="https://colab.research.google.com/github/nepslor/B5203E-TSAF/blob/main/W11/ML_models_and_chronos_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced models for forecasting
In this exercise we will see how to run advanced forecasting models
* A lightgboost model
* [Chronos 2 model](https://arxiv.org/abs/2510.15821)

<img src="https://raw.githubusercontent.com/nepslor/teaching/main/TimeSeriesForecasting/figs/Screenshot_20251201_123940.png" width="900"/>

In [ ]:
!pip install lightgbm wget

In [ ]:

import wget
import zipfile
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Setup path
save_folder = 'jena_climate'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# Download Data
print("Downloading data...")
dat = wget.download('https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip', out=save_folder)

# Extract
with zipfile.ZipFile(dat, 'r') as zip_ref:
    zip_ref.extractall(save_folder)

# Parse Dates
dateparse = lambda x: datetime.strptime(x, '%d.%m.%Y %H:%M:%S')
csv_path = os.path.join(save_folder, 'jena_climate_2009_2016.csv')

# Load & Resample
print("\nProcessing Dataframe...")
df_weather = pd.read_csv(csv_path, parse_dates=['Date Time'], index_col='Date Time', date_parser=dateparse)
# Resample to hourly to make the data manageable
df_weather = df_weather.resample('1h').mean()

# Inspect
print(f"Data loaded. Shape: {df_weather.shape}")
df_weather.head(3)

In [ ]:
original_cols = df_weather.columns

In [ ]:
# @title 1. Data Visualization
print('#'*50)
print('Shape of dataset: {}'.format(df_weather.shape))
print('#'*50)
print('Missing vals: {}'.format(df_weather.isna().sum()))
print('#'*50)
df_weather.iloc[:2000, :].plot(subplots=True, figsize=(20, 10))
plt.show()


In [ ]:
df_weather['T (degC)'].plot(figsize=(20, 4))
df_weather.plot(figsize=(20, 4))

In [ ]:
ma = df_weather.rolling(window=24*7, center=True, min_periods=1).mean()
m_std = df_weather.rolling(window=24*7, center=True, min_periods=1).std()
saliency = (df_weather-ma) / m_std
saliency.plot(figsize=(20, 4));


In [ ]:
outlier = (saliency.abs() > saliency.std()*5) | ((df_weather - df_weather.mean()).abs() > df_weather.std() * 10)

df_weather_noo = df_weather.copy()
# set outliers to nan, interpolate
for variable_key in df_weather.columns:
  df_weather_noo.loc[outlier[variable_key], variable_key] = np.nan
  df_weather_noo[variable_key] = df_weather_noo[variable_key].interpolate(method='pchip')


for variable_key in df_weather.columns:
  outlier_days = np.unique([pd.to_datetime(d.date())  for d  in outlier[variable_key].loc[outlier[variable_key]].index])
  if len(outlier_days) == 0:
    continue
  fig, ax = plt.subplots(len(outlier_days), 1, figsize=(15, len(outlier_days)), layout='constrained')
  ax = np.atleast_1d(ax)
  for a, od in zip(ax.ravel(), outlier_days):
    df_weather[variable_key].loc[df_weather.index>=od].iloc[:200].plot(ax=a)
    df_weather_noo[variable_key].loc[df_weather_noo.index>=od].iloc[:200].plot(ax=a, linestyle='--')
    o_idx = outlier[variable_key].loc[outlier.index>=od].iloc[:200]
    o_idx = o_idx[o_idx].index
    a.scatter(o_idx, df_weather[variable_key].loc[o_idx], color='red')
    a.set_xlabel('')
    a.set_xticks([])
    a.set_xticklabels([])
    a.tick_params(axis='x', bottom=False, labelbottom=False)
    # More aggressive tick removal
    a.xaxis.set_major_locator(plt.NullLocator())
    a.xaxis.set_minor_locator(plt.NullLocator())
    a.text(y=0.9, x=0.9, s=od.strftime('%Y-%m-%d'), ha='right', va='top', transform=a.transAxes, fontsize=12)
  ax[0].set_title(variable_key)


In [ ]:
df_weather_noo.plot(figsize=(20, 4))

In [ ]:
df_weather = df_weather_noo

In [ ]:
# @title 2. Feature generation and embargo split
target = 'T (degC)'
def add_features(df, target = 'T (degC)'):
  df_weather = df.copy()

  df_weather['hour'] = df_weather.index.hour
  df_weather['day_of_week'] = df_weather.index.dayofweek
  df_weather['day_of_year'] = df_weather.index.dayofyear
  df_weather['month'] = df_weather.index.month
  df_weather['year'] = df_weather.index.year

  # Calculate the rolling average of 'T (degC)'
  df_weather[f'{target}_rolling_avg_24h'] = df_weather[target].rolling(window=24, min_periods=1).mean()
  df_weather[f'{target}_rolling_avg_48h'] = df_weather[target].rolling(window=48, min_periods=1).mean()
  df_weather[f'{target}_rolling_avg_72h'] = df_weather[target].rolling(window=72, min_periods=1).mean()
  df_weather[f'{target}_daily_mean_7d'] = df_weather[target].rolling(window=24*7, min_periods=1).mean()
  df_weather[f'{target}_daily_max_7d'] = df_weather[target].rolling(window=24*7, min_periods=1).max()
  df_weather[f'{target}_daily_min_7d'] = df_weather[target].rolling(window=24*7, min_periods=1).min()

  # Create 48 lagged features for 'T (degC)'
  history = {}
  for i in range(1, 24):
      history[f'{target}_lag_{i}'] = df_weather[target].shift(i)



  # Create 24 targets 'T (degC)'
  future = {}
  for i in range(1, 49):
      future[f'{target}_lag_{-i}'] = df_weather[target].shift(-i)

  df_weather = pd.concat([df_weather, pd.DataFrame(history), pd.DataFrame(future)], axis=1)
  df_weather.dropna(inplace=True)
  return df_weather

df_weather = add_features(df_weather_noo, target)
print("Generated new features. Displaying first 5 rows with new columns:")
print(f"New shape of df_weather: {df_weather.shape}")
df_weather.head()


In [ ]:
# @title 3 - Train-test split with embargo
embargo_period = 24*3
n_te = 1000
n_cal = 10000
df_tr = df_weather.iloc[:-(n_te+n_cal+embargo_period)]
df_cal = df_weather.iloc[-(n_te+n_cal+embargo_period):-n_te]
df_te = df_weather.iloc[-(n_te+embargo_period):]

target_names = [c for c in df_weather.columns if target in c and 'lag_-' in c]
x_tr, y_tr = df_tr.drop(columns=target_names), df_tr[target_names]
x_cal, y_cal = df_cal.drop(columns=target_names), df_cal[target_names]
x_te, y_te = df_te.drop(columns=target_names), df_te[target_names]


In [ ]:
from functools import partial
from tqdm import tqdm
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression

def create_pdf(y_cal, y_hat_cal, q_vect = np.arange(11)/10):
  err = y_cal - y_hat_cal
  q_errs = np.quantile(err, q_vect, axis=0).T
  return q_errs

class Multi_reg:
  def __init__(self, model_class) -> None:
     self.model_class = model_class
     self.models = []

  def fit(self, x, y, budget=10000):
    rnd_idx = np.random.choice(x.index, size=budget, replace=False)
    for c in tqdm(y.columns, 'fitting'):
      model = self.model_class()
      model.fit(x.loc[rnd_idx, :], y[c].loc[rnd_idx])
      self.models.append(model)

  def predict(self, x):
    preds = []
    for model in tqdm(self.models, 'predicting'):
      preds.append(model.predict(x))
    return np.array(preds).T

forecas_classes = [LinearRegression, LGBMRegressor]

ys_hat_te = {}
qs_hat_te = {}
for f in forecas_classes:
  m = Multi_reg(partial(f))
  m.fit(x_tr, y_tr)
  y_hat_cal = m.predict(x_cal)
  ys_hat_te[f.__name__] = m.predict(x_te)
  q_errs = create_pdf(y_cal, y_hat_cal, q_vect = np.arange(11)/10)
  qs_hat_te[f.__name__] = np.expand_dims(ys_hat_te[f.__name__], 2) + np.expand_dims(q_errs, 0)



In [ ]:
#@title  Fast Quantile Plot
import plotly.graph_objects as go
import numpy as np

def qs_animation_plotly(y_te, y_hat, qs, n_rows=50, f_name=''):
    """
    y_te:  (n_samples, T)
    y_hat: (n_samples, T)
    qs:    (n_samples, T, n_quantiles)
    """

    t = np.arange(y_te.shape[1])
    n_quant = qs.shape[2]

    # --- INITIAL FRAME ---
    fig = go.Figure()

    # true
    fig.add_trace(go.Scatter(
        x=t, y=y_te[0],
        mode="lines",
        line=dict(color="blue", width=2),
        name="y_te"
    ))

    # predicted
    fig.add_trace(go.Scatter(
        x=t, y=y_hat[0],
        mode="lines",
        line=dict(color="orange", width=2),
        name="y_hat"
    ))

    # quantile curves
    for j in range(n_quant):
        fig.add_trace(go.Scatter(
            x=t, y=qs[0, :, j],
            mode="lines",
            line=dict(color="red", width=1),
            opacity=0.3,
            name=f"q{j}",
            showlegend=False
        ))

    # --- FRAMES ---
    frames = []
    for i in range(n_rows):
        frame_data = [
            go.Scatter(y=y_te[i]),
            go.Scatter(y=y_hat[i]),
        ]
        # quantiles
        frame_data.extend(
            [go.Scatter(y=qs[i, :, j]) for j in range(n_quant)]
        )
        frames.append(go.Frame(data=frame_data, name=f"frame{i}"))

    fig.frames = frames

    # --- LAYOUT WITH PLAY BUTTON ---
    fig.update_layout(
        height=500,
        width=400,
        title="Quantile Forecast Animation for {}".format(f_name),
        updatemenus=[{
            "type": "buttons",
            "buttons": [
                {
                    "label": "Play",
                    "method": "animate",
                    "args": [
                        None,
                        {
                            "frame": {"duration": 50, "redraw": True},
                            "fromcurrent": True,
                            "transition": {"duration": 0}
                        }
                    ]
                }
            ]
        }]
    )

    # y-axis limits
    y_min = min(np.min(y_te), np.min(qs)) - 1
    y_max = max(np.max(y_te), np.max(qs)) + 1
    fig.update_yaxes(range=[y_min, y_max])

    return fig

In [ ]:
keys = list(qs_hat_te.keys())
skip = 24*7
for k in keys:
  display(qs_animation_plotly(y_te.values[skip:], ys_hat_te[k][skip:], qs_hat_te[k][skip:], n_rows=400, f_name=k))


In [ ]:
%pip install 'chronos-forecasting>=2.1' 'pandas[pyarrow]' 'matplotlib'


In [ ]:
x_te_cr = x_te[original_cols].copy().reset_index()
x_te_cr['item_id'] = 'temp'
x_te_cr.head()

In [ ]:
from chronos import BaseChronosPipeline, Chronos2Pipeline

pipeline: Chronos2Pipeline = BaseChronosPipeline.from_pretrained("amazon/chronos-2", device_map="cpu")

In [ ]:
# Generate predictions with covariates
def predict_chronos(x_te_cr, n=400, n_context = 24*7):
  cron_qs=[0.01, 0.1, 0.2, 0.5, 0.7, 0.8, 0.9, 0.99]
  w = np.arange(n_context)
  preds = []
  quantiles = []
  for i in tqdm(range(n)):
    df_context = x_te_cr.loc[w+i]
    cron_pred_df = pipeline.predict_df(
        df_context,
        prediction_length=48,
        quantile_levels=cron_qs,
        id_column="item_id",
        timestamp_column="Date Time",
        target=target,
    )
    preds.append(cron_pred_df['predictions'])
    quantiles.append(cron_pred_df[[str(q) for q in cron_qs]].values)
  y_hat_te = pd.concat(preds, axis=1).T
  q_hat_te = np.rollaxis(np.dstack(quantiles), 2)
  return y_hat_te, q_hat_te

In [ ]:
n_rows = 400 # Number of samples to animate, matching the number of predictions made by Chronos
n_context = 24*7
y_hat_te, q_hat_te = predict_chronos(x_te_cr, n_context = n_context, n=n_rows)
display(qs_animation_plotly(y_te.values[n_context : n_context + n_rows], y_hat_te.values, q_hat_te, n_rows=n_rows, f_name='cronos2'))
